参考：
    1.OpenCV 3计算机视觉 Python语言实现 第二版

在现实生活中，人脸检测可以应用于各行各业，OpenCV提供了人脸检测算法。本部分介绍OpenCV的人脸检测函数，即Haar级联分类器，通过对比分析相邻图像区域来判断给定图像与已知图像是否匹配，将多个Haar级联分类器可以组织成一个层次结构，来识别整体区域（如人脸），而其它分类器可以识别小的区域（如眼睛、鼻子和嘴）。

In [ ]:
#在下面添加此代码以显示单元格中的所有输出
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"

# 获取级联数据

将OpenCV源代码中的副本文件夹data/haarcascades复制到当前项目的文件夹OpenCV/cascades中，该文件夹包含了OpenCV的人脸检测的XML文件，这些文件可用于检测静止图像，视频和摄像头所得到的图像中的人脸。

从文件名可以看到，这些XML文件用于人脸、眼睛、鼻子和嘴的跟踪。这些文件需要正面、直立的人脸图像。

# 使用OpenCV进行人脸检测
## 静态图像中检测人脸
下面代码实现基本的人脸检测：

In [ ]:
import cv2
filename='./OpenCV/big/img_83.jpg'
def detect(filename):
    #声明变量，负责人脸检测
    face_cascade=cv2.CascadeClassifier('./OpenCV/cascades/haarcascade_frontalface_default.xml')
    
    img=cv2.imread(filename)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    #进行实际的人脸检测
    #参数：图像、人脸过程中每次迭代时图像的压缩率、每个人脸矩形保留近邻数目的最小值
    #返回：人脸矩形数目
    faces=face_cascade.detectMultiScale(gray,1.3,5)
    
    for (x,y,w,h) in faces:#依次提取出每个人脸
        img=cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)#直接再图像上绘制矩形
    #cv2.namedWindow('img_83')
    cv2.imshow('img_83',img)
    cv2.imwrite('./OpenCV/img_83.jpg',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
detect(filename)

## 视频中的人脸检测
脚本过程为：打开摄像头、读取帧、检测帧中的人脸、扫描检测到的人脸的眼睛、对人脸绘制蓝色矩形、对眼睛绘制绿色矩形。

In [ ]:

import cv2

#声明变量，负责人脸检测
face_cascade=cv2.CascadeClassifier('./OpenCV/cascades/haarcascade_frontalface_default.xml')
#声明变量，负责眼睛检测
eye_cascade=cv2.CascadeClassifier('./OpenCV/cascades/haarcascade_eye.xml')

cv2.namedWindow("Image")

camera=cv2.VideoCapture(0)
#frame=camera.read()[1]#参数：布尔值，表示是否成功读取帧；读到的帧
while (True):
    k=cv2.waitKey(1) 
    if k == 27:break

    ret,frame=camera.read()#参数：布尔值，表示是否成功读取帧；读到的帧
    
    if ret==False:
        print("没有捕获到视频，ESC键退出")
        break
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #进行实际的人脸检测
    #参数：图像、人脸过程中每次迭代时图像的压缩率、每个人脸矩形保留近邻数目的最小值
    #返回：人脸矩形数目
    faces=face_cascade.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:#依次提取出每个人脸
        img=cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)#直接在图像上绘制矩形
        #在矩形区域检测眼睛
        roi_gray=gray[y:y+h,x:x+w]
        eyes=eye_cascade.detectMultiScale(roi_gray,1.03,5,0,(40,40))
        for (ex,ey,ew,eh) in eyes:#依次提取出眼睛
            cv2.rectangle(frame,(x+ex,y+ey),(x+ex+ew,y+ey+eh),(0,255,0),2)#直接再图像上绘制矩形
    
    cv2.imshow('Image',frame)

    #ret,frame=camera.read()#参数：布尔值，表示是否成功读取帧；读到的帧

camera.release()
cv2.destroyAllWindows()


# 人脸识别
人脸识别就是用程序识别给定图像或视频中的人脸。实现这个目标的方法之一是用一系列分好类的图像（人脸数据库）来训练程序，并给予这些图像来识别。

互联网上有很多人脸数据库可供使用。

下面我们来识别本人，并给出置信度。

## 生成人脸识别数据

视频获取人脸图像，图像满足下面的格式：

    1.灰度格式，后缀是pgm。PGM 是便携式灰度图像格式(portable graymap file format)，在黑白超声图像系统中经常使用PGM格式的图像。PGM格式图像格式分为两类：P2和P5类型。不管是P2还是P5类型的PGM文件，都由两部分组成，文件头部分和数据部分.
    2.图像形状是正方形
    3.图像大小一样。这里使用200*200。


In [3]:
#采集本人人脸图像
import cv2

#声明变量，负责人脸检测
face_cascade=cv2.CascadeClassifier('./OpenCV/cascades/haarcascade_frontalface_default.xml')
#声明变量，负责眼睛检测
#eye_cascade=cv2.CascadeClassifier('./OpenCV/cascades/haarcascade_eye.xml')

cv2.namedWindow("Image")

camera=cv2.VideoCapture(0)
count=0

while (True):
    k=cv2.waitKey(1) 
    if k == 27:break

    ret,frame=camera.read()#参数：布尔值，表示是否成功读取帧；读到的帧
    
    if ret==False:
        print("共捕获%s张图。当前没有捕获到视频，ESC键退出"%str(count))
        break
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #进行实际的人脸检测
    #参数：图像、人脸过程中每次迭代时图像的压缩率、每个人脸矩形保留近邻数目的最小值
    #返回：人脸矩形数目
    faces=face_cascade.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:#依次提取出每个人脸
        img=cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)#直接在图像上绘制矩形
        #在矩形区域检测眼睛
        f=cv2.resize(gray[y:y+h,x:x+w],(200,200))
        cv2.imwrite('./OpenCV/FaceReg/chenl/%s.pgm'%str(count),f)
        count+=1
        
    if count==100:
        break
    
    cv2.imshow('Image',frame)

camera.release()
cv2.destroyAllWindows()

In [4]:
#显示本人，保存的pgm图像
import cv2
count=0
filename='./OpenCV/FaceReg/chenl/'
while (True):   
    img=cv2.imread(filename+str(count)+'.pgm')
    cv2.imshow('pgm',img)
    count+=1
    k=cv2.waitKey(0)
    if (k==27 or count>99):break

cv2.destroyAllWindows()
    

## 人脸识别
OpenCV有三种人脸识别方法，他们分别基于三种不同的算法：Eigenfaces、Fisherfaces、和Local Binary Pattern Histogram（LBPH）。

所有的方法都有类似的过程，即都是用了分好类训练数据集（re你连数据库，每个人都有很多样本）来进行训练，对图像中检测到人脸进行分析，并从两方面确定：是否识别到目标，目标真正被识别的置信度度量（评分）。

## 准备训练数据
生成一个文件，将图像和个人ID对应起来。这里的文件是facereg.csv。

## 加载数据并识别人脸

下面将人脸数据库和CSV文件加载到算法，以训练人脸识别算法。

In [ ]:
#把facereg目录下所有文件转为200*200的pgm人脸文件
import os
import cv2
import numpy as np

def img2pgm(path):
    #声明变量，负责人脸检测
    face_cascade=cv2.CascadeClassifier('./OpenCV/cascades/haarcascade_frontalface_default.xml')

    for dirname,dirnames,filenames in os.walk(path):
        for subdirname in dirnames:
            #print(subdirname)
            subject_path=os.path.join(dirname,subdirname)
            for filename in os.listdir(subject_path):
                if filename.endswith('.pgm'):
                    continue
                im=cv2.imread(os.path.join(subject_path,filename),cv2.IMREAD_GRAYSCALE)
                faces=face_cascade.detectMultiScale(im,1.3,5)

                fcount=0
                for (x,y,w,h) in faces:#依次提取出每个人脸
                    #print(len(faces))        
                    f=cv2.resize(im[y:y+h,x:x+w],(200,200))
                    if len(faces)==1:
                        cv2.imwrite(os.path.join(path,subdirname)
                                +'/'+os.path.splitext(filename)[0]+'.pgm',f)
                    else:
                        cv2.imwrite(os.path.join(path,subdirname)
                                +'/'+os.path.splitext(filename)[0]+'_'+str(fcount)+'.pgm',f)
                    fcount+=1
            
path="./OpenCV/FaceReg/" 
img2pgm(path)

In [6]:
import os
import cv2
import numpy as np

#加载人脸数据库
def read_images(path,sz=None):
    #声明变量，负责人脸检测
    face_cascade=cv2.CascadeClassifier('./OpenCV/cascades/haarcascade_frontalface_default.xml')

    c=0
    X,y=[],[]
    for dirname,dirnames,filenames in os.walk(path):
        for subdirname in dirnames:
            #print(subdirname)
            subject_path=os.path.join(dirname,subdirname)
            for filename in os.listdir(subject_path):
                #print(filename)
                try:
                    if (filename == ".directory" ):
                        continue
                    if (not filename.endswith('.pgm')):#只装载pgm图像
                        continue
                    filepath = os.path.join(subject_path,filename)
                    im=cv2.imread(os.path.join(subject_path,filename),cv2.IMREAD_GRAYSCALE)
                    if (sz is not None):
                        im=cv2.resize(im,(200,200))
                    X.append(np.asarray(im,dtype=np.uint8))
                    y.append(c)
                except IOError:#,(error,strerror):
                    print("I/O error({0}):{1}".format(error,strerror))
                except:
                    print("Unexpected error:",sys.exc_info()[0])
                    raise
            c+=1
                        
    return X,y 
            
path="./OpenCV/FaceReg"   
X,y=read_images(path,sz=(200,200))           
            


## 基于Eigenfaces的人脸识别
Eigenfaces是通过PCA来处理的。PCA本质上是识别某个数据集上的主成分，并计算出训练集相对于数据库的发散程度，并输出一个值。值越小，表明人脸数据库与检测到的人脸之间的差别就小，0表示完全匹配。

下面是测试人脸识别算法的脚本：

In [10]:
import cv2
import numpy as np

path="./OpenCV/FaceReg"   
X,y=read_images(path,sz=(200,200))   

model=cv2.face.EigenFaceRecognizer_create()
model.train(np.asarray(X),np.asarray(y))

camera=cv2.VideoCapture(0)
#声明变量，负责人脸检测
face_cascade=cv2.CascadeClassifier('./OpenCV/cascades/haarcascade_frontalface_default.xml')

cv2.namedWindow("Image")
while (True):
    k=cv2.waitKey(1) 
    if k == 27:break

    ret,frame=camera.read()#参数：布尔值，表示是否成功读取帧；读到的帧
    
    if ret==False:
        print("共捕获%s张图。当前没有捕获到视频，ESC键退出"%str(count))
        break    
    
    #进行实际的人脸检测
    #参数：图像、人脸过程中每次迭代时图像的压缩率、每个人脸矩形保留近邻数目的最小值
    #返回：人脸矩形数目
    faces=face_cascade.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:#依次提取出每个人脸
        img=cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)#直接在图像上绘制矩形
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        roi=gray[x:x+w,y:y+h]
        try:
            roi=cv2.resize(roi,(200,200),interpolation=cv2.INTER_LINEAR)
            params=model.predict(roi)
            print("Label:%s,Confidence:%.2f"%(params[0],params[1]))
            #cv2.putText(img,)
        except:
            continue
    
    cv2.imshow("Image",img)

camera.release()
cv2.destroyAllWindows()

Label:0,Confidence:4761.83
Label:0,Confidence:6081.18
Label:0,Confidence:6057.23
Label:0,Confidence:6108.92
Label:0,Confidence:6515.85
Label:0,Confidence:6461.81
Label:0,Confidence:7042.29
Label:0,Confidence:7120.66
Label:0,Confidence:7488.58
Label:0,Confidence:7680.50
Label:0,Confidence:8121.68
Label:0,Confidence:7934.32
Label:0,Confidence:8581.76
Label:0,Confidence:8689.55
Label:0,Confidence:8507.51
Label:0,Confidence:8307.40
Label:0,Confidence:8079.70
Label:0,Confidence:8090.09
Label:0,Confidence:7716.89
Label:0,Confidence:7659.12
Label:0,Confidence:7224.72
Label:0,Confidence:6856.37
Label:0,Confidence:6746.15
Label:0,Confidence:6128.60
Label:0,Confidence:5752.37
Label:0,Confidence:5362.78
Label:0,Confidence:4761.39
Label:0,Confidence:4389.50
Label:0,Confidence:4414.99
Label:0,Confidence:4262.56
Label:0,Confidence:3407.70
Label:0,Confidence:3108.90
Label:0,Confidence:3648.75
Label:0,Confidence:4544.13
Label:0,Confidence:5267.95
Label:0,Confidence:5440.78
Label:0,Confidence:5758.63
L

可以看到，OpenCV提供了Eigenfaces类，同样提供Fisherfaces、和LBPH类实现人脸识别。